
# **AntiFold for nanobody CDRs (re)design**
## Overview
####[Antifold](https://github.com/oxpig/AntiFold) ([paper link](https://academic.oup.com/bioinformaticsadvances/article/5/1/vbae202/8090019)) is a model trained to reconstruct the sequence of complementarity-determining regions (CDRs) of antibody- or nanobody-antigen complexes, so it resulted in a superior accuracy in the sequence (re)design of these regions with respect to ProteinMPNN, that was trained on a very tiny fraction of loop-mediated interactions
---



In [ ]:
# @title Install AntiFold and Dependencies (please run it twice to solve a glitch)
# @markdown Run this cell first to install all required packages

# Accept Conda Terms of Service first
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r

# Install Miniconda
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

# Initialize conda
!source /usr/local/bin/activate
!conda init bash

# Set up conda environment
!conda create --name antifold python=3.10 -y
# Use the full path to the conda environment to avoid activation issues
!/usr/local/envs/antifold/bin/pip install torch==2.2.0

# Clone and install AntiFold
!git clone https://github.com/oxpig/AntiFold
%cd AntiFold
!/usr/local/envs/antifold/bin/pip install .

# Install additional dependencies for the notebook
!/usr/local/envs/antifold/bin/pip install biopython pandas ipywidgets

# Create necessary directories
!mkdir -p data/pdbs
!mkdir -p output

print("Installation completed successfully!")

In [ ]:
# @title Run AntiFold
# @markdown **Run this cell to upload your PDB file, set parameters and run AntiFold for CDR redesign**

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import os
import subprocess
from google.colab import widgets as colab_widgets

# --- UI Form ---
display(HTML("<h2>Nanobody Design Parameters</h2>"))
nanobody_chain = widgets.Text(description="Nanobody Chain:", value="A")
antigen_chain = widgets.Text(description="Antigen Chain:", value="B", disabled=True) # Antigen is fixed to chain B
num_sequences = widgets.IntSlider(description="Number of sequences:", min=1, max=100, value=10)
seed_value = widgets.Text(description="Random seed:", value="5748")
temperature = widgets.FloatSlider(description="Sampling temperature:", min=0.1, max=1.0, step=0.1, value=0.3)
display(nanobody_chain, antigen_chain, num_sequences, seed_value, temperature)

# --- File Uploader ---
display(HTML("<h3>Upload PDB File</h3>"))
upload_button = widgets.FileUpload(description="Choose PDB file", accept='.pdb', multiple=False)
display(upload_button)

# --- Run Button & Output Area ---
run_button = widgets.Button(description="Run AntiFold", button_style='success')
output_area = widgets.Output()

def get_sequence_from_pdb_manual(pdb_path, chain_id):
    """
    Extracts amino acid sequence from a PDB file without external libraries.
    """
    AA_MAP = {
        'ALA': 'A', 'CYS': 'C', 'ASP': 'D', 'GLU': 'E', 'PHE': 'F',
        'GLY': 'G', 'HIS': 'H', 'ILE': 'I', 'LYS': 'K', 'LEU': 'L',
        'MET': 'M', 'ASN': 'N', 'PRO': 'P', 'GLN': 'Q', 'ARG': 'R',
        'SER': 'S', 'THR': 'T', 'VAL': 'V', 'TRP': 'W', 'TYR': 'Y'
    }
    sequence = []
    seen_residues = set()
    try:
        with open(pdb_path, 'r') as pdb_file:
            for line in pdb_file:
                if line.startswith('ATOM'):
                    line_chain_id = line[21]
                    if line_chain_id == chain_id:
                        residue_id = line[22:27] # Residue seq number + insertion code
                        if residue_id not in seen_residues:
                            seen_residues.add(residue_id)
                            res_name = line[17:20].strip()
                            if res_name in AA_MAP:
                                sequence.append(AA_MAP[res_name])
        return "".join(sequence)
    except FileNotFoundError:
        print(f"⚠️ Error: PDB file not found at {pdb_path}")
        return None
    except Exception as e:
        print(f"⚠️ An error occurred during manual PDB parsing: {e}")
        return None

def on_run_button_clicked(b):
    with output_area:
        clear_output(wait=True)
        print("Starting process...")

        if not upload_button.value:
            print("❌ Please upload a PDB file first.")
            return

        # --- Save Uploaded File ---
        uploaded_file_info = list(upload_button.value.values())[0]
        file_name = list(upload_button.value.keys())[0]
        pdb_path = f"data/pdbs/{file_name}"
        os.makedirs(os.path.dirname(pdb_path), exist_ok=True)
        with open(pdb_path, 'wb') as f:
            f.write(uploaded_file_info['content'])
        print(f"✅ File '{file_name}' uploaded successfully.")

        # --- Extract and Save Sequences ---
        print("\n🔍 Extracting sequences from PDB (dependency-free method)...")
        nb_chain_id = nanobody_chain.value
        ag_chain_id = antigen_chain.value # Always 'B'

        # Extract Nanobody Sequence
        original_nanobody_seq = get_sequence_from_pdb_manual(pdb_path, nb_chain_id)
        if original_nanobody_seq:
            print(f"🧬 Original Nanobody Sequence (Chain {nb_chain_id}): {original_nanobody_seq}")
            with open("output/original_nanobody_sequence.fasta", "w") as f:
                f.write(f">original_nanobody_chain_{nb_chain_id}\n{original_nanobody_seq}\n")
        else:
            print(f"Could not find or process nanobody chain '{nb_chain_id}'.")

        # Extract Antigen Sequence
        antigen_seq = get_sequence_from_pdb_manual(pdb_path, ag_chain_id)
        if antigen_seq:
            print(f"🧬 Antigen Sequence (Chain {ag_chain_id}): {antigen_seq}")
            with open("output/antigen_sequence.fasta", "w") as f:
                f.write(f">antigen_chain_{ag_chain_id}\n{antigen_seq}\n")
        else:
            print(f"Could not find or process antigen chain '{ag_chain_id}'.")


        # --- Run AntiFold Command ---
        print("\n🚀 Running AntiFold... this may take a few minutes.")
        cmd = [
            "/usr/local/envs/antifold/bin/python", "antifold/main.py",
            "--pdb_file", pdb_path,
            "--nanobody_chain", nb_chain_id,
            "--antigen_chain", ag_chain_id,
            "--nanobody_mode",
            "--num_seq_per_target", str(num_sequences.value),
            "--seed", str(seed_value.value),
            "--sampling_temp", str(temperature.value),
            "--out_dir", "output"
        ]
        print(f"▶️ Command: {' '.join(cmd)}")

        try:
            result = subprocess.run(cmd, capture_output=True, text=True, check=True)
            print("\n--- AntiFold STDOUT ---")
            print(result.stdout)
            if result.stderr:
                print("\n--- AntiFold STDERR ---")
                print(result.stderr)
            print("✅ AntiFold completed successfully!")
        except subprocess.CalledProcessError as e:
            print("\n❌ ERROR: AntiFold failed!")
            print("\n--- STDOUT ---")
            print(e.stdout)
            print("\n--- STDERR ---")
            print(e.stderr)
        except Exception as e:
            print(f"An unexpected error occurred: {e}")

run_button.on_click(on_run_button_clicked)
display(run_button, output_area)

In [ ]:
# @title Results and Download (with Antigen Appended)
# @markdown Review and download original, designed, and backup files.

import os
import re
import zipfile
import ipywidgets as widgets
from google.colab import files
from IPython.display import display, HTML

# --- Helper function to create a download button for a specific file ---
def create_download_button(filepath, description, button_style='primary'):
    button = widgets.Button(description=description, button_style=button_style, layout=widgets.Layout(width='auto'))
    def on_click(b):
        try:
            files.download(filepath)
        except Exception as e:
            print(f"Error downloading {os.path.basename(filepath)}: {e}")
    button.on_click(on_click)
    return button

# --- Main Logic ---
output_dir = "output"
display(HTML("<h2>Results Analysis and Download</h2>"))

if not os.path.exists(output_dir):
    display(HTML("<p><b>Output directory not found.</b> Please run the 'Run AntiFold' cell first.</p>"))
else:
    # --- 1. Original and Antigen Sequences ---
    display(HTML("<h3>Original Sequences</h3>"))
    original_files_box = []
    nb_path = os.path.join(output_dir, "original_nanobody_sequence.fasta")
    if os.path.exists(nb_path):
        original_files_box.append(create_download_button(nb_path, "Download Original Nanobody", "info"))
    ag_path = os.path.join(output_dir, "antigen_sequence.fasta")
    if os.path.exists(ag_path):
        original_files_box.append(create_download_button(ag_path, "Download Antigen", "info"))
    if original_files_box:
        display(widgets.HBox(original_files_box))

    # --- 2. Designed Sequences ---
    display(HTML("<hr><h3>Designed Nanobody Sequences</h3>"))
    main_fasta_file = next((f for f in os.listdir(output_dir) if f.endswith('.fasta') and 'original' not in f and 'antigen' not in f), None)

    if main_fasta_file:
        # --- PREPARE ANTIGEN SEQUENCE ---
        antigen_content = ""
        if os.path.exists(ag_path):
            with open(ag_path, 'r') as ag_f:
                antigen_content = ag_f.read().strip()

        # Create a dedicated directory for individual designs
        individual_designs_dir = os.path.join(output_dir, "individual_designs")
        os.makedirs(individual_designs_dir, exist_ok=True)

        # Parse the main fasta file and create individual combined files
        records = []
        with open(os.path.join(output_dir, main_fasta_file), 'r') as f:
            entries = f.read().strip().split('>')
            for i, entry in enumerate(filter(None, entries)):
                lines = entry.strip().split('\n')
                header = lines[0]
                sequence = "".join(lines[1:])
                score_match = re.search(r'score=([\d.-]+)', header)
                score = f"{float(score_match.group(1)):.4f}" if score_match else "N/A"

                filename = f"design_{i}_score_{score}.fasta"
                filepath = os.path.join(individual_designs_dir, filename)

                # Write the combined fasta file
                with open(filepath, 'w') as out_f:
                    # 1. Write the nanobody design
                    out_f.write(f">{header}\n{sequence}\n")
                    # 2. Append the antigen sequence if it exists
                    if antigen_content:
                        out_f.write(f"\n{antigen_content}\n")

                records.append({'filepath': filepath, 'filename': filename, 'score': score})

        # Create a ZIP file of all individual designs as a convenience
        zip_path = os.path.join(output_dir, "all_designed_sequences.zip")
        with zipfile.ZipFile(zip_path, 'w') as zipf:
            for record in records:
                zipf.write(record['filepath'], record['filename'])

        display(HTML("<h4>Download All as a Single Package</h4>"))
        display(create_download_button(zip_path, "Download ALL Designs (ZIP)", "success"))

        # Display individual download toggles
        display(HTML("<h4>Download Individual Sequences (Design + Antigen)</h4>"))

        sorted_records = sorted(records, key=lambda x: float(x['score']) if x['score'] != 'N/A' else -float('inf'), reverse=True)

        ui_list = []
        for record in sorted_records:
            label = widgets.Label(f"File: {record['filename']}", layout=widgets.Layout(width='40%'))
            score_label = widgets.Label(f"Score: {record['score']}", layout=widgets.Layout(width='20%'))
            button = create_download_button(record['filepath'], "Download")
            ui_list.append(widgets.HBox([label, score_label, button]))

        display(widgets.VBox(ui_list))

    else:
        display(HTML("<p>No designed sequence FASTA file was found. Check the output from the previous cell for errors.</p>"))

    # --- 3. Backup Download ---
    display(HTML("<hr><h3>Backup</h3>"))
    backup_zip_path = "output_complete_backup.zip"
    with zipfile.ZipFile(backup_zip_path, 'w') as zipf:
        for root, _, fls in os.walk(output_dir):
            for file in fls:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, output_dir))
    display(create_download_button(backup_zip_path, "Download Complete Output Folder (ZIP)", "warning"))